In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime as dt
import statsmodels as st
import calendar,warnings,itertools,keras,shutil
from datetime import datetime

warnings.filterwarnings('ignore')

#Read data set from environment
dataset=pd.read_csv('DataCoSupplyChainDataset.csv',encoding='unicode_escape',header=0)
#finding shape of dataset
dataset.shape

(180519, 53)

In [2]:
#Null values checking in dataset
dataset.isnull().sum()

Type                                  0
Days for shipping (real)              0
Days for shipment (scheduled)         0
Benefit per order                     0
Sales per customer                    0
Delivery Status                       0
Late_delivery_risk                    0
Category Id                           0
Category Name                         0
Customer City                         0
Customer Country                      0
Customer Email                        0
Customer Fname                        0
Customer Id                           0
Customer Lname                        8
Customer Password                     0
Customer Segment                      0
Customer State                        0
Customer Street                       0
Customer Zipcode                      3
Department Id                         0
Department Name                       0
Latitude                              0
Longitude                             0
Market                                0


In [3]:
#Creating a new dataset instead of drop elements due to in dataset 53 columns but we required only 19

data=pd.DataFrame(dataset,columns=['Type','Benefit per order','Sales per customer','Late_delivery_risk',
                                  'Category Name','Customer Segment','Department Name','Market',
                                  'Order Country','Order Item Discount','Order Item Discount Rate',
                                  'Order Item Product Price','Order Item Profit Ratio','Order Item Quantity','Order Item Total',
                                  'Order Profit Per Order','Order Region','Order Status','Product Price'])

In [4]:
#shape of new dataset
data.shape

(180519, 19)

In [5]:
#Cross verify null values are in new dataset or not
data.isnull().sum()

Type                        0
Benefit per order           0
Sales per customer          0
Late_delivery_risk          0
Category Name               0
Customer Segment            0
Department Name             0
Market                      0
Order Country               0
Order Item Discount         0
Order Item Discount Rate    0
Order Item Product Price    0
Order Item Profit Ratio     0
Order Item Quantity         0
Order Item Total            0
Order Profit Per Order      0
Order Region                0
Order Status                0
Product Price               0
dtype: int64

In [6]:
#Finalize target variable from origin dataset

train_y=pd.DataFrame(dataset,columns=['Sales'])

In [7]:
train_y.head()

,Sales
0,327.75
1,327.75
2,327.75
3,327.75
4,327.75


In [8]:
# Cross verify variable type in data
data.dtypes

Type                         object
Benefit per order           float64
Sales per customer          float64
Late_delivery_risk            int64
Category Name                object
Customer Segment             object
Department Name              object
Market                       object
Order Country                object
Order Item Discount         float64
Order Item Discount Rate    float64
Order Item Product Price    float64
Order Item Profit Ratio     float64
Order Item Quantity           int64
Order Item Total            float64
Order Profit Per Order      float64
Order Region                 object
Order Status                 object
Product Price               float64
dtype: object

In [9]:
df2=pd.DataFrame(data,columns=['Benefit per order','Sales per customer','Late_delivery_risk','Order Item Discount','Order Item Discount Rate',
                              'Order Item Product Price','Order Item Profit Ratio','Order Item Quantity','Order Item Total','Order Profit Per Order',
                              'Product Price'])

In [10]:
df2.shape

(180519, 11)

In [11]:
#creating one more new dataframe for object type variable and we concat later train dataset

df=pd.DataFrame(data,columns=['Type','Category Name','Customer Segment','Department Name','Market','Order Country','Order Region'
                             ,'Order Status'])

In [12]:
#Checking we are created all object varaibles are not
df.dtypes

Type                object
Category Name       object
Customer Segment    object
Department Name     object
Market              object
Order Country       object
Order Region        object
Order Status        object
dtype: object

In [13]:
df.shape

(180519, 8)

In [14]:
#We are doing instaed one hot encode by column wise using pd.get_dummies
df1=pd.get_dummies(df)

In [15]:
df1.head()

,Type_CASH,Type_DEBIT,Type_PAYMENT,Type_TRANSFER,Category Name_Accessories,Category Name_As Seen on TV!,Category Name_Baby,Category Name_Baseball & Softball,Category Name_Basketball,Category Name_Books,...,Order Region_Western Europe,Order Status_CANCELED,Order Status_CLOSED,Order Status_COMPLETE,Order Status_ON_HOLD,Order Status_PAYMENT_REVIEW,Order Status_PENDING,Order Status_PENDING_PAYMENT,Order Status_PROCESSING,Order Status_SUSPECTED_FRAUD
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
df1.shape

(180519, 269)

In [17]:
df1= df1.astype(np.float32)

In [18]:
df2= df2.astype(np.float32)

In [19]:
merge=[df1,df2]
train_x=pd.concat(merge,axis=1)

In [20]:
train_x.shape

(180519, 280)

In [21]:
#Till now we completed EDA process next process is model building

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU 

In [22]:
model=Sequential()

In [23]:
n_cols=train_x.shape[1]
leaky_relu_alpha = 0.1

In [24]:
#model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(Dense(100,activation='relu',input_shape=(n_cols,)))
model.add(Dense(100,activation='relu'))
model.add(Dense(1))

In [25]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [26]:
model.fit(train_x,train_y,validation_split=0.2,epochs=10)

Epoch 1/10
4513/4513 [==============================] - 10s 2ms/step - loss: 2.3457e-05 - accuracy: 0.0000e+00 - val_loss: 2.7631e-05 - val_accuracy: 0.0000e+00
Epoch 2/10
4513/4513 [==============================] - 10s 2ms/step - loss: 2.3457e-05 - accuracy: 0.0000e+00 - val_loss: 2.7631e-05 - val_accuracy: 0.0000e+00
Epoch 3/10
4513/4513 [==============================] - 10s 2ms/step - loss: 2.3457e-05 - accuracy: 0.0000e+00 - val_loss: 2.7631e-05 - val_accuracy: 0.0000e+00
Epoch 4/10
4513/4513 [==============================] - 10s 2ms/step - loss: 2.3457e-05 - accuracy: 0.0000e+00 - val_loss: 2.7631e-05 - val_accuracy: 0.0000e+00
Epoch 5/10
4513/4513 [==============================] - 10s 2ms/step - loss: 2.3457e-05 - accuracy: 0.0000e+00 - val_loss: 2.7631e-05 - val_accuracy: 0.0000e+00
Epoch 6/10
4513/4513 [==============================] - 10s 2ms/step - loss: 2.3457e-05 - accuracy: 0.0000e+00 - val_loss: 2.7631e-05 - val_accuracy: 0.0000e+00
Epoch 7/10
4513/4513 [============